In [32]:
from keras.models import Sequential
from keras.layers import ConvLSTM2D, Conv3D, Dense, Flatten, GlobalAveragePooling2D, Conv2D, BatchNormalization, Reshape
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

In [5]:
h5f = h5py.File('./data/data_2018.h5','r')

In [6]:
data = h5f['total_2018'][:]
data_gen = h5f['gen_2018'][:]
h5f.close()

In [7]:
data.shape, data_gen.shape

((8760, 7, 75, 75), (8760,))

In [22]:
data = np.rollaxis(data, 1, 4)

In [23]:
data.shape

(8760, 75, 75, 7)

In [ ]:
plt.matshow(np.flipud(data[0,:,:,1]))

In [7]:
def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback, data.shape[-3], data.shape[-2], data.shape[-1]))
        targets = np.zeros((len(rows), data.shape[-3], data.shape[-2], data.shape[-1]))
        
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j])
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets

In [24]:
def generator(data, target, lookback, delay, min_index, max_index, shuffle=False, batch_size=128):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while True:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        samples = np.zeros((len(rows), lookback, data.shape[-3], data.shape[-2], data.shape[-1]))
        targets = np.zeros((len(rows)))
        
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j])
            samples[j] = data[indices]
            targets[j] = target[rows[j] + delay - 1]
        yield samples, targets

In [25]:
train_gen = generator(data, data_gen, 72, 0, 0, None, shuffle=True, batch_size=5)

In [33]:
model = Sequential()
model.add(ConvLSTM2D(filters=21, kernel_size=(3, 3),
                   input_shape=(None, 75, 75, 7),
                    data_format='channels_last', return_sequences=True,
                   padding='same'))
model.add(BatchNormalization())

model.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
                    data_format='channels_last',
                   padding='same'))

model.add(Reshape((-1, 40)))

model.add(Dense(units=1))

model.compile(optimizer='rmsprop', loss='mae')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_12 (ConvLSTM2D) (None, None, 75, 75, 21)  21252     
_________________________________________________________________
batch_normalization_12 (Batc (None, None, 75, 75, 21)  84        
_________________________________________________________________
conv3d_10 (Conv3D)           (None, None, 75, 75, 1)   568       
_________________________________________________________________
reshape_1 (Reshape)          (None, None, 40)          0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 1)           41        
Total params: 21,945
Trainable params: 21,903
Non-trainable params: 42
_________________________________________________________________


In [10]:
test_gen = generator(data, 24, 1, 0, None, shuffle=True, batch_size=5)

In [14]:
history = model.fit_generator(test_gen,
                              steps_per_epoch=10,
                             epochs=10)

Epoch 1/10


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv_lst_m2d_2/convolution}}]]
	 [[{{node loss_1/mul}}]]